# Integrantes
- Aguilar Martínez Erick Yair
- Martínez Muñoz Alan Magno
- Mendoza Hernández Carlos Emiliano

Implementar el cálculo de los estimadores:

$$
\hat{\mathbf{\beta}}=(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

$$
\sigma^2_\epsilon = \frac{\sum_{i=1}^n(y_i-\hat{y}_i)^2}{n-q}
$$

$$
\text{var}(\hat{\mathbf{\beta}})=\sigma_\epsilon^2(\mathbf{X}^T\mathbf{X})^{-1}
$$